<a href="https://colab.research.google.com/github/enesdouaydin/HER-SEYDEN-BIRAZ-BIRAZ/blob/main/distilitation1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install --upgrade transformers accelerate bitsandbytes


In [11]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio


object address  : 0x7ae4fbaf9720
object refcount : 2
object type     : 0x9d7580
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C
^C


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM # veya AutoModelForSequenceClassification

# ---- AYARLAR ----
# Hugging Face'deki model kimliğinizi buraya yazın
model_kimligi = "GOOLGMAN/turk-hukuk-modeli" # Kendi model kimliğinizle güncelleyin
# ---- AYARLAR BİTTİ ----

try:
    # Adım 1: Modeli ve Tokenizer'ı Yükleme
    print(f"'{model_kimligi}' modeli Hugging Face Hub'dan indiriliyor...")
    tokenizer = AutoTokenizer.from_pretrained(model_kimligi, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_kimligi, trust_remote_code=True)

    # Adım 2: 'device' Değişkenini Tanımlama (HATAYI ÇÖZEN KISIM)
    # GPU varsa "cuda", yoksa "cpu" kullanmasını söylüyoruz.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Modeli tespit edilen cihaza (GPU/CPU) gönderiyoruz.
    model.to(device)

    print(f"\nModel başarıyla yüklendi ve '{device}' üzerinde çalışmaya hazır!")

    # Adım 3: Soru Sorma Fonksiyonunu Tanımlama
    def hukuki_soru_sor(soru, sistem_mesaji=None):
        """
        Fine-tune edilmiş hukuk modeline bir soru sorar ve cevabını yazdırır.
        """
        if sistem_mesaji is None:
            sistem_mesaji = "Sen, Türkiye Cumhuriyeti kanunları, yönetmelikleri ve hukuki kavramlar hakkında bilgi veren bir yapay zeka asistanısın. Yalnızca sorulan hukuki sorulara, kanun metinlerine dayanarak cevap ver. Cevapların bilgilendiricidir ve yasal tavsiye niteliği taşımaz."

        mesajlar = [
            {"role": "system", "content": sistem_mesaji},
            {"role": "user", "content": soru}
        ]
        prompt = tokenizer.apply_chat_template(mesajlar, tokenize=False, add_generation_prompt=True)

        # Girdiyi token'larına ayırıp doğru cihaza gönderiyoruz.
        # Fonksiyon artık dışarıda tanımlanan 'device' değişkenini görebilir.
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        print(f"\nSoru: {soru}")
        print("\n--- Modelin Cevabı ---")

        outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
        cevap_tokenlari = outputs[0][inputs["input_ids"].shape[1]:]
        cevap_metni = tokenizer.decode(cevap_tokenlari, skip_special_tokens=True)

        print(cevap_metni)
        print("----------------------")

    # Adım 4: Modeli Test Etme
    hukuki_soru_sor("Temyiz başvurusu için yasal süre nedir?")
    hukuki_soru_sor("Kat mülkiyeti kanununa göre yönetici nasıl seçilir?")

except Exception as e:
    print(f"\nBir hata oluştu: {e}")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_kimligi = "GOOLGMAN/turk-hukuk-modeli"

tokenizer = AutoTokenizer.from_pretrained(model_kimligi, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_kimligi, trust_remote_code=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def hukuki_soru_sor(soru):
    sistem_mesaji = "Sen, Türkiye Cumhuriyeti kanunları hakkında bilgi veren bir yapay zeka asistanısın..."
    mesajlar = [
        {"role": "system", "content": sistem_mesaji},
        {"role": "user", "content": soru}
    ]
    prompt = tokenizer.apply_chat_template(mesajlar, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    print(f"\nSoru: {soru}")
    print("\n--- Modelin Cevabı ---")

    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    cevap_tokenlari = outputs[0][inputs["input_ids"].shape[1]:]
    cevap_metni = tokenizer.decode(cevap_tokenlari, skip_special_tokens=True)

    print(cevap_metni)
    print("----------------------")

hukuki_soru_sor("Anayasa, Türk Vatanı ve Milletinin ebedi varlığını nasıl güvence altına almayı amaçlıyor?")



In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:

input_csv_path = "/content/drive/MyDrive/turkish_law_dataset.csv" # Buraya dosyanızın tam yolunu yapıştırın

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from torch.nn import CrossEntropyLoss, KLDivLoss
from torch.nn.functional import log_softmax, softmax
from datasets import load_dataset, Dataset # Gerçek veri setleri için datasets kütüphanesi
import pandas as pd # CSV okumak için
import os # Dosya yolları için

# --- AYARLAR ---
# Öğretmen modelinizin Hugging Face kimliği (Qwen 1.5 4B fine-tune edilmiş hali)
teacher_model_id = "GOOLGMAN/turk-hukuk-modeli"

# Öğrenci modelinizin temel Hugging Face kimliği (Qwen 1.5 1.8B)
# Bu model daha küçüktür ve distilasyon için temel alınacaktır.
student_base_model_id = "Qwen/Qwen1.5-1.8B"

# Distile edilmiş öğrenci modelinizin kaydedileceği Hugging Face kimliği
# Kendi Hugging Face kullanıcı adınızla güncelleyin!
distilled_model_name = "YOUR_HUGGINGFACE_USERNAME/turk-hukuk-modeli-distilled-1.8B-qwen"

# Google Drive'daki CSV dosyanızın tam yolu
input_csv_path = "/content/drive/MyDrive/turkish_law_dataset.csv"

# CSV dosyanızdaki metin verilerini içeren sütunun adı
# Eğer farklıysa, burayı güncelleyin (örn. "Question", "Text", "Soru" vb.)
text_column_name = "text" # Varsayılan olarak 'text' sütununu arayacak

# Distilasyon Hyperparametreleri
alpha = 0.5       # Süpervize kayıp ile distilasyon kaybı arasındaki ağırlık (0.0 ile 1.0 arası)
temperature = 2.0 # Yumuşatma sıcaklığı (genellikle 2.0-5.0 arası iyi başlar)

# Eğitim Argümanları
num_train_epochs = 1            # Eğitim epoch sayısı (veri setinizin büyüklüğüne göre artırabilirsiniz)
per_device_train_batch_size = 2 # GPU/CPU başına batch boyutu (GPU belleğinize göre ayarlayın)
gradient_accumulation_steps = 8 # Küçük batch boyutunda gradient biriktirme adımları (per_device_train_batch_size * gradient_accumulation_steps = etkili batch boyutu)
learning_rate = 5e-5            # Öğrenme oranı
output_dir = "./distillation_results" # Eğitim çıktıları ve modelin kaydedileceği dizin
# --- AYARLAR BİTTİ ---

try:
    print("--- Distilasyon Süreci Başlatılıyor ---")

    # 1. Modelleri ve Tokenizer'ları Yükleme
    print(f"'{teacher_model_id}' öğretmen modeli indiriliyor...")
    # Qwen modelleri için trust_remote_code=True gereklidir
    tokenizer = AutoTokenizer.from_pretrained(teacher_model_id, trust_remote_code=True)
    teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_id, trust_remote_code=True)

    print(f"'{student_base_model_id}' öğrenci temel modeli indiriliyor...")
    # Öğrenci modeli için Qwen 1.5 1.8B temel modelini yüklüyoruz
    student_model = AutoModelForCausalLM.from_pretrained(student_base_model_id, trust_remote_code=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    teacher_model.to(device)
    student_model.to(device)
    teacher_model.eval() # Öğretmen modeli eğitim modunda olmamalıdır

    print(f"\nModeller başarıyla yüklendi: Öğretmen '{teacher_model_id}' ve Öğrenci '{student_base_model_id}' '{device}' üzerinde çalışmaya hazır!")

    # 2. Veri Setini Hazırlama (Google Drive'dan CSV)
    print(f"\nVeri seti '{input_csv_path}' adresinden okunuyor...")

    if not os.path.exists(input_csv_path):
        raise FileNotFoundError(f"'{input_csv_path}' adresinde dosya bulunamadı. Google Drive'ınızın bağlı olduğundan ve yolun doğru olduğundan emin olun.")

    # CSV dosyasını pandas ile oku
    df = pd.read_csv(input_csv_path)

    # DataFrame'i datasets formatına dönüştür
    # Sadece belirtilen metin sütununu kullanarak Dataset oluşturun
    if text_column_name not in df.columns:
        raise ValueError(f"CSV dosyasında '{text_column_name}' adında bir sütun bulunamadı. Lütfen 'text_column_name' değişkenini doğru sütun adıyla güncelleyin. Mevcut sütunlar: {df.columns.tolist()}")

    # Sadece metin sütununu alıp bir liste haline getirin
    raw_texts = df[text_column_name].tolist()

    # Hugging Face Dataset objesine dönüştür
    # Eğer modeliniz chat formatında fine-tune edildiyse, bu kısmı ayarlamanız gerekebilir.
    # Örneğin, her satır için {"role": "user", "content": text} yapısına dönüştürme.
    # Şimdilik basitçe metinleri kullanıyoruz.
    train_dataset = Dataset.from_dict({"text": raw_texts})

    print(f"Veri seti başarıyla yüklendi. Toplam {len(train_dataset)} örnek var.")
    print("İlk 3 örnek:")
    for i, example in enumerate(train_dataset[:3]["text"]):
        print(f"  {i+1}. {example[:150]}...") # İlk 150 karakteri göster

    # Veri setini token'larına ayırın (padding ve truncation ile)
    def tokenize_function(examples):
        # Qwen tokenizer'ı için özel chat template uygulaması, eğer modeliniz bu şekilde eğitildiyse
        # Eğer teacher modeliniz doğrudan metin tamamlama için fine-tune edildiyse, bu adımı atlayabilirsiniz.
        # Örneğin, aşağıdaki gibi yapabilirsiniz:
        # messages = [{"role": "user", "content": examples["text"]}]
        # return tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt", add_generation_prompt=True)
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

    # Tokenizasyon işlemini haritalayın
    tokenized_datasets = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=train_dataset.column_names # Orijinal sütunları kaldırır
    )
    # Etiketler genellikle input_ids'in kaydırılmış hali olacağı için, burada labels sütununu oluşturmuyoruz.
    # Trainer'ın compute_loss metodunda bu işlem yapılacak.

    # 3. Özel Distilasyon Eğiticisini Tanımlama
    class CustomDistillationTrainer(Trainer):
        def __init__(self, teacher_model, *args, alpha=0.5, temperature=2.0, **kwargs):
            super().__init__(*args, **kwargs)
            self.teacher_model = teacher_model
            self.alpha = alpha  # Süpervize kayıp ile distilasyon kaybı arasındaki ağırlık
            self.temperature = temperature # Yumuşatma sıcaklığı
            self.kl_loss = KLDivLoss(reduction="batchmean")

        def compute_loss(self, model, inputs, return_outputs=False):
            # Öğrenci modelinin çıktıları
            student_outputs = model(**inputs, output_hidden_states=True, output_attentions=True)
            student_logits = student_outputs.logits

            # Öğretmen modelinin çıktıları (gradient hesaplamasını engellemek için no_grad içinde olmalı)
            with torch.no_grad():
                teacher_outputs = self.teacher_model(**inputs, output_hidden_states=True, output_attentions=True)
                teacher_logits = teacher_outputs.logits

            # Hesaplanan loss:
            # LLM'lerde genellikle next token prediction olduğu için labels input_ids'in kaydırılmış hali olabilir
            labels = inputs["input_ids"].clone()

            # Tokenları kaydırın ki, token < n, n'yi tahmin etsin
            shift_logits_student = student_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Geçerli tokenların dışında kalan kısımları ignore etmek için
            # -100 genellikle ignore_index olarak kullanılır
            # Eğer Qwen'in tokenizer'ı farklı bir pad token ID kullanıyorsa kontrol edin
            if tokenizer.pad_token_id is not None:
                shift_labels = torch.where(shift_labels == tokenizer.pad_token_id, -100, shift_labels)

            loss_fct = CrossEntropyLoss(ignore_index=-100) # Pad tokenları kayıptan hariç tut
            hard_loss = loss_fct(shift_logits_student.view(-1, student_logits.size(-1)), shift_labels.view(-1))

            # Distilasyon (soft target) loss
            # log_softmax ve softmax uygulayın ve sıcaklık (temperature) kullanın.
            # Sıcaklığın karesiyle ölçekleme, Hinton'ın orijinal makalesindeki formülü takip eder.
            # Distilasyon kaybında da pad tokenları ignore etmek önemlidir.
            active_logits_student = student_logits[..., :-1, :][shift_labels != -100]
            active_logits_teacher = teacher_logits[..., :-1, :][shift_labels != -100]

            if active_logits_student.numel() == 0: # Eğer hiç aktif token yoksa (örn. boş batch)
                distillation_loss = torch.tensor(0.0, device=student_logits.device)
            else:
                distillation_loss = self.kl_loss(
                    log_softmax(active_logits_student / self.temperature, dim=-1),
                    softmax(active_logits_teacher / self.temperature, dim=-1)
                ) * (self.temperature ** 2)

            # Toplam kayıp: Süpervize ve distilasyon kayıplarının ağırlıklı toplamı
            total_loss = self.alpha * hard_loss + (1 - self.alpha) * distillation_loss

            return (total_loss, student_outputs) if return_outputs else total_loss

    # 4. Eğitim Argümanlarını Ayarlama
    training_args = TrainingArguments(
        output_dir=output_dir, # Eğitim çıktıları ve modelin kaydedileceği dizin
        num_train_epochs=num_train_epochs, # Eğitim epoch sayısı
        per_device_train_batch_size=per_device_train_batch_size, # Her cihaz için batch boyutu
        gradient_accumulation_steps=gradient_accumulation_steps, # Gradient biriktirme adımları
        logging_dir="./distillation_logs", # Logların kaydedileceği dizin
        logging_steps=100, # Loglama sıklığı (her 100 adımda bir)
        save_strategy="epoch", # Her epoch sonunda modeli kaydet
        learning_rate=learning_rate, # Öğrenme oranı
        report_to="none", # Varsayılan raporlama araçlarına göndermeyi kapatın
        push_to_hub=True, # Eğitimin sonunda modeli Hugging Face Hub'a itmek için
        hub_model_id=distilled_model_name, # Hugging Face Hub'daki model kimliği
    )

    # 5. Eğiticiyi Başlatma
    trainer = CustomDistillationTrainer(
        model=student_model,
        teacher_model=teacher_model,
        args=training_args,
        train_dataset=tokenized_datasets, # Tokenize edilmiş veri setini kullan
        tokenizer=tokenizer,
        alpha=alpha, # Süpervize ve distilasyon kaybı arasındaki ağırlık
        temperature=temperature, # Distilasyon sıcaklığı
    )

    print("\nDistilasyon eğitimi başlıyor...")
    trainer.train()
    print("\nDistilasyon eğitimi tamamlandı.")

    # 6. Distile Edilmiş Modeli Kaydetme ve Hugging Face Hub'a Yükleme
    print(f"\nDistile edilmiş model '{distilled_model_name}' Hugging Face Hub'a yükleniyor...")
    # trainer.push_to_hub() zaten push_to_hub=True ayarlandığı için eğitim sonunda otomatik olarak çalışır.
    # Eğer otomatik yükleme başarısız olursa veya manuel yüklemek isterseniz:
    student_model.save_pretrained(distilled_model_name.split('/')[-1]) # Yerel olarak da kaydedin
    tokenizer.save_pretrained(distilled_model_name.split('/')[-1])

    # Hugging Face Hub'a manuel yükleme (isteğe bağlı, push_to_hub=True zaten yapar)
    # from huggingface_hub import HfApi
    # api = HfApi()
    # api.upload_folder(
    #     folder_path=distilled_model_name.split('/')[-1],
    #     repo_id=distilled_model_name,
    #     repo_type="model",
    # )
    print(f"\nDistile edilmiş model '{distilled_model_name}' dizinine kaydedildi ve Hugging Face Hub'a yükleme işlemi başlatıldı (veya tamamlandı).")

    # Distile edilmiş modeli test etme (yeni yükleyerek)
    print("\nDistile edilmiş model test ediliyor...")
    # Hugging Face Hub'dan distile edilmiş modeli yükleyin
    distilled_tokenizer = AutoTokenizer.from_pretrained(distilled_model_name, trust_remote_code=True)
    distilled_model = AutoModelForCausalLM.from_pretrained(distilled_model_name, trust_remote_code=True).to(device)

    def hukuki_soru_sor_distilled(soru, model_to_use, tokenizer_to_use, device_to_use, sistem_mesaji=None):
        """
        Distile edilmiş hukuk modeline bir soru sorar ve cevabını yazdırır.
        """
        if sistem_mesaji is None:
            sistem_mesaji = "Sen, Türkiye Cumhuriyeti kanunları, yönetmelikleri ve hukuki kavramlar hakkında bilgi veren bir yapay zeka asistanısın. Yalnızca sorulan hukuki sorulara, kanun metinlerine dayanarak cevap ver. Cevapların bilgilendiricidir ve yasal tavsiye niteliği taşımaz."

        mesajlar = [
            {"role": "system", "content": sistem_mesaji},
            {"role": "user", "content": soru}
        ]
        # Qwen tokenizer'ı için apply_chat_template kullanımı önemlidir.
        prompt = tokenizer_to_use.apply_chat_template(mesajlar, tokenize=False, add_generation_prompt=True)

        # Girdiyi token'larına ayırıp doğru cihaza gönderiyoruz.
        inputs = tokenizer_to_use(prompt, return_tensors="pt").to(device_to_use)

        print(f"\nSoru (Distile Edilmiş Model): {soru}")
        print("\n--- Distile Edilmiş Modelin Cevabı ---")

        # Modelin generate metodunu kullanarak cevap üretin
        # Qwen modelleri için pad_token_id genellikle eos_token_id ile aynıdır.
        outputs = model_to_use.generate(**inputs, max_new_tokens=512, do_sample=False, pad_token_id=tokenizer_to_use.eos_token_id)

        # Üretilen token'lardan sadece yeni olanları alın ve decode edin
        cevap_tokenlari = outputs[0][inputs["input_ids"].shape[1]:]
        cevap_metni = tokenizer_to_use.decode(cevap_tokenlari, skip_special_tokens=True)

        print(cevap_metni)
        print("----------------------")

    # Distile edilmiş modeli test etme
    hukuki_soru_sor_distilled("Temyiz başvurusu için yasal süre nedir?", distilled_model, distilled_tokenizer, device)
    hukuki_soru_sor_distilled("Kat mülkiyeti kanununa göre yönetici nasıl seçilir?", distilled_model, distilled_tokenizer, device)
    hukuki_soru_sor_distilled("Boşanma davasında iştirak nafakası ne demektir?", distilled_model, distilled_tokenizer, device)

except FileNotFoundError as fnf_error:
    print(f"\nDosya hatası: {fnf_error}")
    print("Lütfen Google Drive bağlantınızı ve dosya yolunu kontrol edin.")
except ValueError as val_error:
    print(f"\nDeğer hatası: {val_error}")
    print("Lütfen CSV dosyanızın yapısını ve 'text_column_name' ayarını kontrol edin.")
except Exception as e:
    print(f"\nBeklenmeyen bir hata oluştu: {e}")

--- Distilasyon Süreci Başlatılıyor ---
'GOOLGMAN/turk-hukuk-modeli' öğretmen modeli indiriliyor...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
import pandas as pd

# Orijinal CSV dosyanızın yolu
input_csv_path = "/content/drive/MyDrive/turkish_law_dataset.csv"

# Dönüştürülmüş CSV dosyasının kaydedileceği yol
output_csv_path = "/content/drive/MyDrive/turkish_law_dataset_processed.csv"

# CSV dosyanızdaki soru ve cevap sütunlarının adları
prompt_col = "soru"
completion_col = "cevap"

try:
    # CSV dosyasını oku
    df = pd.read_csv(input_csv_path)

    # Gerekli sütunların varlığını kontrol et
    if prompt_col not in df.columns or completion_col not in df.columns:
        print(f"Hata: '{prompt_col}' veya '{completion_col}' sütunları CSV dosyasında bulunamadı.")
        print(f"Mevcut sütunlar: {df.columns.tolist()}")
    else:
        # 'soru' ve 'cevap' sütunlarını birleştirerek 'text' sütunu oluştur
        # Modelin daha iyi anlaması için bir format kullanın (örn: "Soru: ...\nCevap: ...")
        df['text'] = df[prompt_col].astype(str) + "\nCevap: " + df[completion_col].astype(str)

        # Sadece 'text' sütununu içeren yeni bir DataFrame oluştur
        # Diğer sütunları isterseniz tutabilirsiniz, ancak distilasyon için 'text' yeterlidir.
        processed_df = df[['text']]

        # Yeni CSV dosyasını kaydet (index olmadan)
        processed_df.to_csv(output_csv_path, index=False)
        print(f"CSV dosyası başarıyla dönüştürüldü ve '{output_csv_path}' konumuna kaydedildi.")
        print("İlk 5 satır:")
        print(processed_df.head())

except FileNotFoundError:
    print(f"Hata: '{input_csv_path}' adresinde dosya bulunamadı. Google Drive bağlantınızı ve yolun doğru olduğundan emin olun.")
except Exception as e:
    print(f"Beklenmeyen bir hata oluştu: {e}")

CSV dosyası başarıyla dönüştürüldü ve '/content/drive/MyDrive/turkish_law_dataset_processed.csv' konumuna kaydedildi.
İlk 5 satır:
                                                text
0  Anayasa, Türk Vatanı ve Milletinin ebedi varlı...
1  Anayasa, Türkiye Cumhuriyetinin hangi milliyet...
2  Anayasa, Türkiye Cumhuriyetini hangi konumda t...
3  Anayasa, Türkiye Cumhuriyetinin hangi hedefler...
4  Anayasa, egemenliğin kime ait olduğunu nasıl b...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from torch.nn import CrossEntropyLoss, KLDivLoss
from torch.nn.functional import log_softmax, softmax
from datasets import load_dataset, Dataset # Gerçek veri setleri için datasets kütüphanesi
import pandas as pd # CSV okumak için
import os # Dosya yolları için
import gc # Garbage Collector için

# PEFT kütüphanesi için importlar
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Gerekli kütüphanelerin yüklü olduğundan emin olun ve yükleyin
# Bu blok, kod çalıştırıldığında kütüphanelerin mevcut olmasını sağlar.
required_packages = ["bitsandbytes", "accelerate", "datasets", "transformers", "pandas", "peft"]
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"'{package}' kütüphanesi bulunamadı. Yükleniyor...")
        # Google Colab veya Jupyter ortamında iseniz '!' kullanın
        # Yerel Python ortamında iseniz '!' işaretini kaldırın.
        os.system(f"pip install -q {package}")
        print(f"'{package}' başarıyla yüklendi.")

# --- BELLEK OPTİMİZasyonu İÇİN ORTAM DEĞİŞKENİ ---
# PyTorch'un bellek parçalanmasını azaltmasına yardımcı olur.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- AYARLAR ---
# Öğretmen modelinizin Hugging Face kimliği (fine-tune edilmiş Qwen 1.5 4B modeliniz)
teacher_model_id = "GOOLGMAN/turk-hukuk-modeli"

# Öğrenci modelinizin temel Hugging Face kimliği (Qwen 1.5 1.8B)
# Bu model daha küçüktür ve distilasyon için temel alınacaktır.
student_base_model_id = "Qwen/Qwen1.5-1.8B"

# Distile edilmiş öğrenci modelinizin kaydedileceği Hugging Face kimliği
# KENDİ HUGGING FACE KULLANICI ADINIZLA GÜNCELLEYİN!
# Örnek: "your_username/turk-hukuk-modeli-distilled-1.8B-qwen"
distilled_model_name = "GOOLGMAN/turk-hukuk-modeli-distilled-1.8B-qwen"

# Google Drive'daki CSV dosyanızın tam yolu
# Google Colab kullanıyorsanız, Drive'ınızı bağladığınızdan emin olun.
# Bu dosya artık doğrudan 'text' sütununu içermelidir.
input_csv_path = "/content/drive/MyDrive/turkish_law_dataset_processed.csv" # YENİ PROCESSED CSV DOSYASI KULLANILDI

# CSV dosyanızdaki metin verilerini içeren sütunun adı.
# Artık doğrudan 'text' sütununu arıyoruz.
text_column_name = "text"

# Distilasyon Hyperparametreleri
alpha = 0.5       # Süpervize kayıp ile distilasyon kaybı arasındaki ağırlık (0.0 ile 1.0 arası)
temperature = 2.0 # Yumuşatma sıcaklığı (genellikle 2.0-5.0 arası iyi başlar)

# Eğitim Argümanları
num_train_epochs = 1            # Eğitim epoch sayısı (veri setinizin büyüklüğüne göre artırabilirsiniz)
per_device_train_batch_size = 1 # **BELLEK OPTİMİZASYONU İÇİN KRİTİK: GPU başına batch boyutu 1**
gradient_accumulation_steps = 64 # **BELLEK OPTİMİZASYONU İÇİN KRİTİK: Etkili batch boyutunu korumak için**
learning_rate = 5e-5            # Öğrenme oranı
output_dir = "./distillation_results" # Eğitim çıktıları ve modelin kaydedileceği dizin

# Maksimum sekans uzunluğu (bellek tüketimini etkiler)
# Eğer metinleriniz genellikle 512 token'dan kısaysa, daha da azaltabilirsiniz.
max_sequence_length = 256 # **BELLEK OPTİMİZASYONU İÇİN AZALTILDI: 512'den 256'ya**
# --- AYARLAR BİTTİ ---

try:
    print("--- Distilasyon Süreci Başlatılıyor ---")
    print("\n" * 3) # Daha fazla görsel ayrım
    print("#####################################################################################")
    print("##                                 ÇOK ÖNEMLİ UYARI!                               ##")
    print("##                                                                                 ##")
    print("## 'CUDA out of memory' hatası alırsanız, LÜTFEN Google Colab çalışma zamanını     ##")
    print("## (Runtime -> Restart runtime) YENİDEN BAŞLATIN.                                  ##")
    print("##                                                                                 ##")
    print("## Bu hata genellikle GPU belleğinin başka işlemler tarafından zaten dolu olmasından ##")
    print("## kaynaklanır ve yeniden başlatma bu belleği temizler.                            ##")
    print("#####################################################################################")
    print("\n" * 3)

    # 8-bit kuantizasyon yapılandırması
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    # LoRA (PEFT) yapılandırması
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # 1. Modelleri ve Tokenizer'ları Yükleme
    print(f"'{teacher_model_id}' öğretmen modeli indiriliyor ve 8-bit olarak yükleniyor...")
    tokenizer = AutoTokenizer.from_pretrained(teacher_model_id, trust_remote_code=True)

    teacher_model = AutoModelForCausalLM.from_pretrained(
        teacher_model_id,
        trust_remote_code=True,
        quantization_config=bnb_config
    )
    teacher_model = prepare_model_for_kbit_training(teacher_model)
    teacher_model = get_peft_model(teacher_model, lora_config)
    teacher_model.eval()
    torch.cuda.empty_cache()
    gc.collect()

    print(f"'{student_base_model_id}' öğrenci temel modeli indiriliyor ve 8-bit olarak yükleniyor...")
    student_model = AutoModelForCausalLM.from_pretrained(
        student_base_model_id,
        trust_remote_code=True,
        quantization_config=bnb_config
    )
    student_model = prepare_model_for_kbit_training(student_model)
    student_model = get_peft_model(student_model, lora_config)
    torch.cuda.empty_cache()
    gc.collect()

    print(f"\nModeller başarıyla yüklendi: Öğretmen '{teacher_model_id}' ve Öğrenci '{student_base_model_id}' (8-bit kuantize edilmiş ve PEFT adaptörleri eklendi) hazır!")

    # 2. Veri Setini Hazırlama (Google Drive'dan CSV)
    print(f"\nVeri seti '{input_csv_path}' adresinden okunuyor...")

    if not os.path.exists(input_csv_path):
        raise FileNotFoundError(f"'{input_csv_path}' adresinde dosya bulunamadı. Google Drive'ınızın bağlı olduğundan ve yolun doğru olduğundan emin olun.")

    # CSV dosyasını pandas ile oku
    df = pd.read_csv(input_csv_path)

    # Doğrudan 'text' sütununun varlığını kontrol et
    if text_column_name not in df.columns:
        raise ValueError(f"CSV dosyasında '{text_column_name}' sütunu bulunamadı. Lütfen CSV dosyanızın bu sütunu içerdiğinden emin olun. Mevcut sütunlar: {df.columns.tolist()}")

    # 'text' sütunundaki metinleri al
    raw_texts = df[text_column_name].astype(str).tolist()

    train_dataset = Dataset.from_dict({"text": raw_texts})

    print(f"Veri seti başarıyla yüklendi. Toplam {len(train_dataset)} örnek var.")
    print("İlk 3 örnek:")
    for i, example in enumerate(train_dataset[:3]["text"]):
        print(f"  {i+1}. {example[:150]}...") # İlk 150 karakteri göster

    # Veri setini token'larına ayırın (padding ve truncation ile)
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_sequence_length)

    tokenized_datasets = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=train_dataset.column_names
    )
    torch.cuda.empty_cache()
    gc.collect()

    # 3. Özel Distilasyon Eğiticisini Tanımlama
    class CustomDistillationTrainer(Trainer):
        def __init__(self, teacher_model, *args, alpha=0.5, temperature=2.0, **kwargs):
            super().__init__(*args, **kwargs)
            self.teacher_model = teacher_model
            self.alpha = alpha
            self.temperature = temperature
            self.kl_loss = KLDivLoss(reduction="batchmean")

        # compute_loss metodunu **kwargs alacak şekilde güncelledik
        def compute_loss(self, model, inputs, return_outputs=False, **kwargs): # **kwargs eklendi
            student_outputs = model(**inputs, output_hidden_states=True, output_attentions=True)
            student_logits = student_outputs.logits.to(model.device)

            with torch.no_grad():
                teacher_outputs = self.teacher_model(**inputs, output_hidden_states=True, output_attentions=True)
                teacher_logits = teacher_outputs.logits.to(model.device)

            labels = inputs["input_ids"].clone().to(model.device)

            shift_logits_student = student_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            if tokenizer.pad_token_id is not None:
                shift_labels = torch.where(shift_labels == tokenizer.pad_token_id, -100, shift_labels)

            loss_fct = CrossEntropyLoss(ignore_index=-100)
            hard_loss = loss_fct(shift_logits_student.view(-1, student_logits.size(-1)), shift_labels.view(-1))

            active_logits_student = student_logits[..., :-1, :][shift_labels != -100]
            active_logits_teacher = teacher_logits[..., :-1, :][shift_labels != -100]

            if active_logits_student.numel() == 0:
                distillation_loss = torch.tensor(0.0, device=model.device)
            else:
                distillation_loss = self.kl_loss(
                    log_softmax(active_logits_student / self.temperature, dim=-1),
                    softmax(active_logits_teacher / self.temperature, dim=-1)
                ) * (self.temperature ** 2)

            total_loss = self.alpha * hard_loss + (1 - self.alpha) * distillation_loss

            return (total_loss, student_outputs) if return_outputs else total_loss

    # 4. Eğitim Argümanlarını Ayarlama
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        logging_dir="./distillation_logs",
        logging_steps=100,
        save_strategy="epoch",
        learning_rate=learning_rate,
        report_to="none",
        push_to_hub=True,
        hub_model_id=distilled_model_name,
        # Bellek Optimizasyonları
        fp16=False,
        bf16=True,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={'use_reentrant': False},
    )

    # 5. Eğiticiyi Başlatma
    trainer = CustomDistillationTrainer(
        model=student_model,
        teacher_model=teacher_model,
        args=training_args,
        train_dataset=tokenized_datasets,
        processing_class=tokenizer, # tokenizer yerine processing_class kullanıldı
        alpha=alpha,
        temperature=temperature,
    )

    print("\nDistilasyon eğitimi başlıyor...")
    trainer.train()
    print("\nDistilasyon eğitimi tamamlandı.")

    # 6. Distile Edilmiş Modeli Kaydetme ve Hugging Face Hub'a Yükleme
    print(f"\nDistile edilmiş model '{distilled_model_name}' Hugging Face Hub'a yükleniyor...")
    # PEFT adaptörlerini kaydet
    trainer.model.save_pretrained(distilled_model_name.split('/')[-1])
    tokenizer.save_pretrained(distilled_model_name.split('/')[-1])

    print(f"\nDistile edilmiş model '{distilled_model_name}' dizinine kaydedildi ve Hugging Face Hub'a yükleme işlemi başlatıldı (veya tamamlandı).")

    # Distile edilmiş modeli test etme (yeni yükleyerek)
    print("\nDistile edilmiş model test ediliyor...")
    # PEFT adaptörlerini yüklemek için önce temel modeli yükleyin, sonra adaptörleri uygulayın.
    distilled_tokenizer = AutoTokenizer.from_pretrained(distilled_model_name, trust_remote_code=True)
    distilled_model = AutoModelForCausalLM.from_pretrained(
        student_base_model_id, # Temel öğrenci modelini yükle
        trust_remote_code=True,
        quantization_config=bnb_config # Kuantizasyon yapılandırmasını uygula
    )
    # Yüklenen temel modele PEFT adaptörlerini uygula
    distilled_model = prepare_model_for_kbit_training(distilled_model)
    distilled_model = get_peft_model(distilled_model, lora_config)

    # Kaydedilen adaptörleri yükle
    distilled_model.load_adapter(distilled_model_name.split('/')[-1])

    # Modelin cihazını otomatik olarak ayarlaması beklenir, ancak emin olmak için:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    distilled_model.to(device)

    def hukuki_soru_sor_distilled(soru, model_to_use, tokenizer_to_use, device_to_use, sistem_mesaji=None):
        """
        Distile edilmiş hukuk modeline bir soru sorar ve cevabını yazdırır.
        """
        if sistem_mesaji is None:
            sistem_mesaji = "Sen, Türkiye Cumhuriyeti kanunları, yönetmelikleri ve hukuki kavramlar hakkında bilgi veren bir yapay zeka asistanısın. Yalnızca sorulan hukuki sorulara, kanun metinlerine dayanarak cevap ver. Cevapların bilgilendiricidir ve yasal tavsiye niteliği taşımaz."

        mesajlar = [
            {"role": "system", "content": sistem_mesaji},
            {"role": "user", "content": soru}
        ]
        prompt = tokenizer_to_use.apply_chat_template(mesajlar, tokenize=False, add_generation_prompt=True)

        inputs = tokenizer_to_use(prompt, return_tensors="pt").to(device_to_use)

        print(f"\nSoru (Distile Edilmiş Model): {soru}")
        print("\n--- Distile Edilmiş Modelin Cevabı ---")

        outputs = model_to_use.generate(**inputs, max_new_tokens=512, do_sample=False, pad_token_id=tokenizer_to_use.eos_token_id)

        cevap_tokenlari = outputs[0][inputs["input_ids"].shape[1]:]
        cevap_metni = tokenizer_to_use.decode(cevap_tokenlari, skip_special_tokens=True)

        print(cevap_metni)
        print("----------------------")

    # Distile edilmiş modeli test etme
    hukuki_soru_sor_distilled("Temyiz başvurusu için yasal süre nedir?", distilled_model, distilled_tokenizer, device)
    hukuki_soru_sor_distilled("Kat mülkiyeti kanununa göre yönetici nasıl seçilir?", distilled_model, distilled_tokenizer, device)
    hukuki_soru_sor_distilled("Boşanma davasında iştirak nafakası ne demektir?", distilled_model, distilled_tokenizer, device)

except FileNotFoundError as fnf_error:
    print(f"\nDosya hatası: {fnf_error}")
    print("Lütfen Google Drive bağlantınızı ve dosya yolunu kontrol edin.")
except ValueError as val_error:
    print(f"\nDeğer hatası: {val_error}")
    print("Lütfen CSV dosyanızın yapısını ve 'text_column_name' ayarını kontrol edin.")
except Exception as e:
    print(f"\nBeklenmeyen bir hata oluştu: {e}")


--- Distilasyon Süreci Başlatılıyor ---




#####################################################################################
##                                 ÇOK ÖNEMLİ UYARI!                               ##
##                                                                                 ##
## 'CUDA out of memory' hatası alırsanız, LÜTFEN Google Colab çalışma zamanını     ##
## (Runtime -> Restart runtime) YENİDEN BAŞLATIN.                                  ##
##                                                                                 ##
## Bu hata genellikle GPU belleğinin başka işlemler tarafından zaten dolu olmasından ##
## kaynaklanır ve yeniden başlatma bu belleği temizler.                            ##
#####################################################################################




'GOOLGMAN/turk-hukuk-modeli' öğretmen modeli indiriliyor ve 8-bit olarak yükleniyor...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


'Qwen/Qwen1.5-1.8B' öğrenci temel modeli indiriliyor ve 8-bit olarak yükleniyor...


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'unsloth/Qwen3-4B-Base' to 'Qwen/Qwen1.5-1.8B'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


...

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Distilasyon eğitimi başlıyor...


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during

Step,Training Loss
100,3.155200


Görüntülenen çıkış son 5000 satıra kısaltıldı.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs wil